In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_theme()

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
pd.set_option('display.max_columns', None)

## Load data

In [4]:
insurance = pd.read_csv("../data/2016_us_insurance.csv", dtype={"STATEFP20": object})
unplanned_births = pd.read_csv("../data/unplanned_births.csv", dtype={"STATEFP20": object})
family_planning = pd.read_csv("../data/family_planning.csv", dtype={"STATEFP20": object})
contraceptives = pd.read_csv("../data/contraceptive_use.csv", dtype={"STATEFP20": object})
childcare = pd.read_csv("../data/abortion-childcare.csv")
paid_leave = pd.read_csv("../data/abortion-paid-leave.csv")
adoption_num = pd.read_csv("../data/adoption_numbers_statewide.csv")
adoption_prop = pd.read_csv("../data/adoption_proportion_statewide.csv")
parental_leave = pd.read_csv("../data/parental_leave_rankings.csv")

In [5]:
insurance.head()

,year,state_name,STATEFP20,agecat,racecat,sexcat,iprcat,NUI,NIC
0,2016,Alabama,01,Under 65 years,All,Both,All,427972.0,3545106.0
1,2016,Alabama,01,Under 65 years,All,Both,<=200%,274697.0,1239595.0
2,2016,Alabama,01,Under 65 years,All,Both,<=250%,319119.0,1557484.0
3,2016,Alabama,01,Under 65 years,All,Both,<=138%,201733.0,846280.0
4,2016,Alabama,01,Under 65 years,All,Both,<=400%,386721.0,2340032.0


In [6]:
unplanned_births.head()

,state,perc_funded,fed_cost,num_funded,indv_cost,state_cost,total_cost,year,STATEFP20
0,Alabama,61.6,250.5,18200,336,72.6,323.2,2010,01
1,Alaska,64.3,70.8,3000,790,42.9,113.7,2010,02
2,Arizona,64.6,509.4,24200,531,161.5,670.9,2010,04
3,Arkansas,72.3,266.8,13800,576,61.9,328.7,2010,05
4,California,64.3,1062.0,105300,222,689.3,1751.0,2010,06


In [7]:
family_planning.head()

,state,perc_need_met16,num_served16,num_u20_served16,num_centers15,total_expend15,STATEFP20
0,Alabama,28,97600,20300,187,69742000,01
1,Alaska,54,21640,4750,157,10158000,02
2,Arizona,19,88140,16500,232,57560000,04
3,Arkansas,23,50960,10610,163,11397000,05
4,California,64,1618010,252840,1697,454706000,06


In [8]:
contraceptives.head()

,state,using,any,fem_steril,male_steril,implant,iud,pill,nonlarc_horm,condoms,withdrawal,other,none,year,STATEFP20
0,Alabama,71.5,75.9,19.3,4.7,NaN,5.4,20.6,NaN,17.1,NaN,4.2,24.1,2017,01
1,Alaska,76.5,82.7,10.6,NaN,NaN,20.6,10.2,NaN,24.2,NaN,NaN,17.3,2017,02
2,Arizona,64.2,67.4,12.3,4.1,2.2,9.3,14.7,NaN,16.5,1.1,6.3,32.6,2017,04
3,Arkansas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,05
4,California,72.0,76.6,9.1,1.3,3.5,9.4,19.7,3.1,22.7,2.3,5.4,23.4,2017,06


In [9]:
childcare.head()

,Unnamed: 0,State,Abortion Overview,Child Care Total Score
0,0,Alabama,significantly restricted,1.42
1,1,Alaska,neither,1.37
2,2,Arizona,significantly restricted,0.97
3,3,Arkansas,significantly restricted,1.54
4,4,California,protected,0.95


In [10]:
paid_leave.head()

,Unnamed: 0,State,Abortion Overview,Paid Leave Legislation Total Score
0,0,Alabama,significantly restricted,0.0
1,1,Alaska,neither,0.0
2,2,Arizona,significantly restricted,0.0
3,3,Arkansas,significantly restricted,0.0
4,4,California,protected,2.0


In [11]:
adoption_num.head()

,Unnamed: 0,State,Waiting for Adoption 2010,Waiting for Adoption 2011,Waiting for Adoption 2012,Waiting for Adoption 2013,Waiting for Adoption 2014,Waiting for Adoption 2015,Waiting for Adoption 2016,Waiting for Adoption 2017,Waiting for Adoption 2018,Waiting for Adoption 2019,Parental Rights Terminated 2010,Parental Rights Terminated 2011,Parental Rights Terminated 2012,Parental Rights Terminated 2013,Parental Rights Terminated 2014,Parental Rights Terminated 2015,Parental Rights Terminated 2016,Parental Rights Terminated 2017,Parental Rights Terminated 2018,Parental Rights Terminated 2019,Adopted 2010,Adopted 2011,Adopted 2012,Adopted 2013,Adopted 2014,Adopted 2015,Adopted 2016,Adopted 2017,Adopted 2018,Adopted 2019,Served 2010,Served 2011,Served 2012,Served 2013,Served 2014,Served 2015,Served 2016,Served 2017,Served 2018,Served 2019
0,0,Alabama,1271,1297,1156,1084,1044,1020,1075,1268,1346,1688,757,701,543,615,573,461,421,533,514,523,606,447,587,532,544,512,503,504,712,738,8119,8395,7907,7322,7520,7989,8381,9012,9512,9741
1,1,Alaska,696,720,805,866,704,783,949,1032,1006,951,381,389,387,399,366,356,425,432,430,375,336,293,309,328,367,357,312,366,353,396,2750,2669,2727,2842,3109,3659,3868,4085,4032,4216
2,2,Arizona,2673,2822,2914,3579,3641,4060,4713,4749,4216,3876,2174,2220,2299,2692,2841,3006,3379,3295,2966,2687,2045,2275,2275,2522,3090,3236,3654,4298,3926,3373,17110,18142,21267,23854,26253,28557,29090,26897,23909,22689
3,3,Arkansas,1604,1414,1023,994,1034,1122,1198,1330,1351,1182,812,742,543,642,757,757,795,795,809,787,597,591,703,691,743,740,758,971,1001,984,7474,7506,7513,7411,7211,7759,8285,8545,7811,7230
4,4,California,14892,14630,13136,15667,14096,14180,14689,15097,14866,14436,7230,6928,6021,6562,6379,6617,6712,7107,7098,7262,6459,5710,5938,5541,5471,6079,6542,6524,6966,6981,92254,86563,84820,86255,87087,87484,85896,80408,78215,77889


In [12]:
adoption_prop.head()

,Unnamed: 0,State,Adopted Over Waiting for Adoption 2010,Parental Rights Terminated Over Waiting For Adoption 2010,Adopted Over Waiting for Adoption 2011,Parental Rights Terminated Over Waiting For Adoption 2011,Adopted Over Waiting for Adoption 2012,Parental Rights Terminated Over Waiting For Adoption 2012,Adopted Over Waiting for Adoption 2013,Parental Rights Terminated Over Waiting For Adoption 2013,Adopted Over Waiting for Adoption 2014,Parental Rights Terminated Over Waiting For Adoption 2014,Adopted Over Waiting for Adoption 2015,Parental Rights Terminated Over Waiting For Adoption 2015,Adopted Over Waiting for Adoption 2016,Parental Rights Terminated Over Waiting For Adoption 2016,Adopted Over Waiting for Adoption 2017,Parental Rights Terminated Over Waiting For Adoption 2017,Adopted Over Waiting for Adoption 2018,Parental Rights Terminated Over Waiting For Adoption 2018,Adopted Over Waiting for Adoption 2019,Parental Rights Terminated Over Waiting For Adoption 2019
0,0,Alabama,0.476790,0.595594,0.344641,0.540478,0.507785,0.469723,0.490775,0.567343,0.521073,0.548851,0.501961,0.451961,0.467907,0.391628,0.397476,0.420347,0.528975,0.381872,0.437204,0.309834
1,1,Alaska,0.482759,0.547414,0.406944,0.540278,0.383851,0.480745,0.378753,0.460739,0.521307,0.519886,0.455939,0.454662,0.328767,0.447840,0.354651,0.418605,0.350895,0.427435,0.416404,0.394322
2,2,Arizona,0.765058,0.813318,0.806166,0.786676,0.780714,0.788950,0.704666,0.752165,0.848668,0.780280,0.797044,0.740394,0.775302,0.716953,0.905033,0.693830,0.931214,0.703510,0.870227,0.693240
3,3,Arkansas,0.372195,0.506234,0.417963,0.524752,0.687195,0.530792,0.695171,0.645875,0.718569,0.732108,0.659537,0.674688,0.632721,0.663606,0.730075,0.597744,0.740933,0.598816,0.832487,0.665821
4,4,California,0.433723,0.485496,0.390294,0.473548,0.452040,0.458359,0.353673,0.418842,0.388124,0.452540,0.428702,0.466643,0.445367,0.456941,0.432139,0.470756,0.468586,0.477465,0.483583,0.503048


In [13]:
parental_leave.head()

,Unnamed: 0,State,Score,Grade,Abortion Overview
0,0,California,135,A-,protected
1,1,Hawaii,100,B+,protected
2,2,Oregon,95,B+,protected
3,3,Connecticut,85,B,protected
4,4,New Jersey,85,B,neither
